In [1]:
using DynamicPolynomials
using TSSOS
using MAT
using LinearAlgebra
using Printf
using JuMP

using Distributed

In [2]:
@polyvar temp1[1:4, 1:4]
@polyvar temp2[1:4, 1:4]
temp_p = temp2 * 1.1 + temp1 * 1.1

function Joint2SE3(c, s, x, y, z)
    ## we always rotate about the z axis
    T = temp_p * 0
    T[1,1] = c
    T[2,2] = c
    T[1,2] = -s
    T[2,1] = s
    T[3,3] = 1
    
    T[1,4] = c * x - s * y
    T[2,4] = s * x + c * y
    T[3,4] = z
    T[4,4] = 1
    return T
end

function SE3_Euler_num(y, p, r, lx, ly, lz)
    T = temp_p * 0
    T[1:3, 1:3] = rotz_num(y) * roty_num(p) * rotx_num(r)
    T[4,4] = 1.0
    T[1,4] = lx
    T[2,4] = ly
    T[3,4] = lz
    return T
end

function rotx_num(t)
    R = temp_p[1:3, 1:3] * 0
    R[2,2] = cos(t)
    R[3,3] = cos(t)
    R[2,3] = -sin(t)
    R[3,2] = sin(t)
    R[1,1] = 1.0
    return R
end

function roty_num(t)
    R = temp_p[1:3, 1:3] * 0
    R[1,1] = cos(t)
    R[3,3] = cos(t)
    R[1,3] = sin(t)
    R[3,1] = -sin(t)
    R[2,2] = 1.0
    return R
end

function rotz_num(t)
    R = temp_p[1:3, 1:3] * 0
    R[1,1] = cos(t)
    R[2,2] = cos(t)
    R[1,2] = -sin(t)
    R[2,1] = sin(t)
    R[3,3] = 1.0
    return R
end

function quat2rot_num(w, x, y, z)
    R = temp_p[1:3, 1:3] * 0
    nq = sqrt(w^2 + x^2 + y^2 + z^2)
    w = w / nq
    x = x / nq
    y = y / nq
    z = z / nq
    
    R[1,1] = 1 - 2 * (y^2 + z^2)
    R[2,2] = 1 - 2 * (x^2 + z^2)
    R[3,3] = 1 - 2 * (x^2 + y^2)
    
    R[1,2] = 2 * (x * y - w * z)
    R[1,3] = 2 * (w * y + x * z)
    R[2,3] = 2 * (y * z - w * x)
    
    R[2,1] = 2 * (x * y + w * z)
    R[3,1] = 2 * (- w * y + x * z)
    R[3,2] = 2 * (y * z + w * x)
    return R
end

function cross_(x, y) 
    z = temp_p[1:3, 1]
    z[1] = -x[3] * y[2] + x[2] * y[3]
    z[2] =  x[3] * y[1] - x[1] * y[3]
    z[3] = -x[2] * y[1] + x[1] * y[2]
    return z
end

function add_SO3_cons(R, eq)
    T = R' * R
#     print(T[1,1] - 1.0)
    append!(eq, [T[1,1] - 1.0])
    append!(eq, [T[2,2] - 1.0])
    append!(eq, [T[3,3] - 1.0])
    append!(eq, [T[1,2]])
    append!(eq, [T[1,3]])
    append!(eq, [T[2,3]])
    
    append!(eq, cross_(R[1,:], R[2, :]) - R[3,:])
    append!(eq, cross_(R[2,:], R[3, :]) - R[1,:])
    append!(eq, cross_(R[3,:], R[1, :]) - R[2,:])
    
    return eq
end

function add_SE3_tran_cons(T1, T2, eq)
    T = T1 - T2
    
    append!(eq, [T[1, 1]])
    append!(eq, [T[1, 2]])
    append!(eq, [T[1, 3]])
    
    
    append!(eq, [T[2, 1]])
    append!(eq, [T[2, 2]])
    append!(eq, [T[2, 3]])
    
    
    append!(eq, [T[3, 1]])
    append!(eq, [T[3, 2]])
    append!(eq, [T[3, 3]])
    
    append!(eq, [T[1, 4]])
    append!(eq, [T[2, 4]])
    append!(eq, [T[3, 4]])    
    return eq
end

function add_SO3_tran_cons(T1, T2, eq)
    T = T1 - T2
    
    append!(eq, [T[1, 1]])
    append!(eq, [T[1, 2]])
    append!(eq, [T[1, 3]])
    
    
    append!(eq, [T[2, 1]])
    append!(eq, [T[2, 2]])
    append!(eq, [T[2, 3]])
    
    
    append!(eq, [T[3, 1]])
    append!(eq, [T[3, 2]])
    append!(eq, [T[3, 3]])
    
    return eq
end


function make_SE3(R, x, y, z)
    T = temp_p * 0
    T[1:3, 1:3] = R
    T[1,4] = x
    T[2,4] = y
    T[3,4] = z
    T[4,4] = 1.0
    return T
end

function SE3_inverse(T)
    Tnew = T
    Tnew[1:3, 1:3] = T[1:3, 1:3]'
    Tnew[1:3, 4] = - T[1:3, 1:3]' * T[1:3, 4]
    return Tnew
end

# Joint2SE3(c[1], s[1], x[1], y[1], z[1])
# R = quat2rot_num(randn(), randn(), randn(), randn())
# R * R'
# rotz_num(c[1,1])
# T = SE3_Euler_num(randn(), randn(), randn(), randn(), randn(), randn())
# cross(R[1, 1,:], R[1, 2, :])

###########################################
function DH1(c, s)
    ## we always rotate about the z axis
    T = temp_p * 0
    T[1,1] = c
    T[2,2] = c
    T[1,2] = -s
    T[2,1] = s
    T[3,3] = 1
    
    T[1,4] = 0
    T[2,4] = 0
    T[3,4] = 0
    
    T[4,4] = 1
    return T
end

function DH2(c, s)
    ## we always rotate about the z axis
    T = temp_p * 0
    T[1,1] = c
    T[1,2] = -s
    T[3,2] = -c
    T[3,1] = -s
    T[2,3] = 1
    
    T[1,4] = 0.
    T[2,4] = 0
    T[3,4] = 0
    
    T[4,4] = 1
    return T
end

function DH3(c, s)
    ## we always rotate about the z axis
    T = temp_p * 0
    T[1,1] = c
    T[2,2] = c
    T[1,2] = -s
    T[2,1] = s
    T[3,3] = 1
        
    T[1,4] = 431.8 / 1000
    T[2,4] = 0
    T[3,4] = 124.46 / 1000
    
    T[4,4] = 1
    return T
end

function DH4(c, s)
    ## we always rotate about the z axis
    T = temp_p * 0
    T[1,1] = c
    T[1,2] = -s
    T[3,2] = -c
    T[3,1] = -s
    T[2,3] = 1

    
    T[1,4] = 20.32 / 1000
    T[2,4] = 431.8 / 1000
    T[3,4] = 0
    
    T[4,4] = 1
    return T
end

function DH5(c, s)
    ## we always rotate about the z axis
    T = temp_p * 0
    T[1,1] = c
    T[1,2] = -s
    T[3,2] = c
    T[3,1] = s
    T[2,3] = -1

    
    T[1,4] = 0
    T[2,4] = 0.
    T[3,4] = 0
    
    T[4,4] = 1
    return T
end

function DH6(c, s)
    ## we always rotate about the z axis
    T = temp_p * 0
    T[1,1] = c
    T[1,2] = -s
    T[3,2] = -c
    T[3,1] = -s
    T[2,3] = 1

    
    T[1,4] = 0
    T[2,4] = 0.
    T[3,4] = 0
    
    T[4,4] = 1
    return T
end


DH6 (generic function with 1 method)

In [4]:
puma_goal = matread("puma_test_data.mat")
R_list = puma_goal["R_list"]
p_list = puma_goal["p_list"]

joint_list = puma_goal["sol_list"]


1×14553 Matrix{Any}:
 [-1.72701 -0.475237 … 0.848254 1.53816]  …  Matrix{Float64}(undef, 0, 0)

In [6]:
function puma_batch_test_hard(T_end)
    
#     ub = [60, 45, 75, 135, 100, 180] * pi / 180
#     lb = [-60, -225, -250, -135, -100, -180] * pi / 180
    
#     ub = [180, 45, 75, 135, 100, 180] * pi / 180
#     lb = [-180, -225, -250, -135, -100, -180] * pi / 180

    ub = [170, 45, 75, 135, 100, 180] * pi / 180
    lb = [-170, -225, -250, -135, -100, -180] * pi / 180

    dof = 6
    
    @polyvar R[1:dof+1, 1:3, 1:3] 
    @polyvar c[1:dof]
    @polyvar s[1:dof]
    @polyvar x[1:dof+1]
    @polyvar y[1:dof+1]
    @polyvar z[1:dof+1]
    var = append!(reshape(R, (1+dof) * 9), c, s, x, y, z)
    # print(var)

    eq = [temp_p[1,1] * 0]
    ineq = []

    eq[1] = R[1, 1, 1] - 1.0
    append!(eq, [R[1, 1, 2] - 0.0])
    append!(eq, [R[1, 1, 3] - 0.0])
    append!(eq, [R[1, 2, 1] - 0.0])
    append!(eq, [R[1, 2, 2] - 1.0])
    append!(eq, [R[1, 2, 3] - 0.0])
    append!(eq, [R[1, 3, 1] - 0.0])
    append!(eq, [R[1, 3, 2] - 0.0])
    append!(eq, [R[1, 3, 3] - 1.0])

    append!(eq, [x[1] - 0.0])
    append!(eq, [y[1] - 0.0])
    append!(eq, [z[1] - 0.0])

    f_act = 0

    for k = 1:dof

        append!(eq, [c[k]^2 + s[k]^2 - 1]) # joint SO2 constrains
        eq = add_SO3_cons(R[k+1,:,:], eq) # SO3 constraints

        Tk = make_SE3(R[k,:,:], x[k], y[k], z[k])
        Tkp1 = make_SE3(R[k+1,:,:], x[k+1], y[k+1], z[k+1])

        if k == 1
            Act = DH1(c[k], s[k])
            mid = (ub[k] + lb[k]) / 2
        end
        if k == 2
            Act = DH2(c[k], s[k])
        end
        if k == 3
            Act = DH3(c[k], s[k])
        end
        if k == 4
            Act = DH4(c[k], s[k])
        end
        if k == 5
            Act = DH5(c[k], s[k])
        end
        if k == 6
            Act = DH6(c[k], s[k])
        end
        
        mid = (ub[k] + lb[k]) / 2
        r = (ub[k] - lb[k]) / 2
        
        append!(ineq, [c[k] * cos(mid) + s[k] * sin(mid) - cos(r)])  # cos(t - t0) >= cos(r)

        eq = add_SE3_tran_cons(Tkp1, Tk * Act, eq)
        # append!(ineq, [100 - (x[k+1]^2 + y[k+1]^2 + z[k+1]^2)])
        
        f_act = f_act + (1 - c[k])^2 + s[k]^2
    end

    

    R_goal = T_end[1:3, 1:3] 

    R_diff = R[end, :, :] - R_goal
    R_diff = R_diff' * R_diff
    f_R = R_diff[1,1] + R_diff[2,2] + R_diff[3,3]
    f_p = (x[end] - T_end[1, 4])^2 + (y[end] - T_end[2, 4])^2 + (z[end] - T_end[3, 4])^2
    
    for i = 1:3
        for j = 1:3
            append!(eq, [ R_goal[i,j] - R[end, i, j] ])
        end
    end
    
    append!(eq, [ x[end] - T_end[1, 4] ])
    append!(eq, [ y[end] - T_end[2, 4] ])
    append!(eq, [ z[end] - T_end[3, 4] ])
    

    f = f_act # + f_R + f_p # * 1e-5 # 1e-3 * sum(var.^2)
    pop = append!([f * 1.0], ineq, eq)


    eq_len = length(eq)

    order = 2
    
    time_1_start = time()
    # you may need to modify this function to export the model information 
    # opt_1,sol,data,model,sol_approx = cs_tssos_first(pop, var, order, numeq=eq_len, CS="NC", TS=false, MomentOne=true, solution=true, QUIET=false);
    opt_1,sol,data = cs_tssos_first(pop, var, order, numeq=eq_len, CS="NC", TS=false, MomentOne=true, solution=true, QUIET=false);
    time_1_end = time()
    
    elapsed_1 = time_1_end - time_1_start
    
    moment = []
    for k = 1:length(data.moment)
        append!(moment, [convert(Matrix{Float64}, data.moment[k])]) # data.Mmatrix[k])
    end

    Tg = zeros(4, 4)
    for i = 1:4
        for j = 1:4
            Tg[i,j] = T_end[i, j]
        end
    end
    
    # flag = primal_status(model)
    # flag = string(flag)
    
    # flag2 = termination_status(model)
    # flag2 = string(flag2)
    
    log = Dict("sol" => sol, "dof" => dof, "moment" => moment, # "sol_approx" => sol_approx,
        "elapsed" => elapsed_1, "Tg" => Tg, #"flag"=>string(flag),
        "opt"=>opt_1) #"flag2"=>flag2)

    return log
end

puma_goal = matread("puma_test_data.mat")
R_list = puma_goal["R_list"]
p_list = puma_goal["p_list"]
for id = 1:3682
    T_end = zeros(4, 4)
    T_end[1:3, 1:3] = R_list[id, :, :]
    T_end[1:3, end] = p_list[id, :]
    T_end[end, end] = 1
    log = puma_batch_test_hard(T_end)
    file_name = "MD_puma_"  *string(id) * ".mat"
#     matwrite(file_name, log)
end

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[9, 8, 6, 2, 4, 3]
[6, 11, 18, 30, 9, 6]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.1036687 seconds. The maximal size of cliques is 9.
Assembling the SDP...
There are 9762 affine constraints.
SDP assembling time: 0.9006447 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 9762            
  Cones                  : 0               
  Scalar variables       : 9916            
  Matrix variables       : 115             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear